In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import copy

In [2]:
options = Options()
options.page_load_strategy = 'normal'
service = Service(executable_path="C:/Users/prasa/Downloads/chromedriver_win32/chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)

In [3]:
driver.get('https://quotes.carwow.co.uk/')
email_field = driver.find_element(By.CLASS_NAME, 'input-full-width')
email_field.send_keys('sathasivamprasanta@gmail.com')
password_field = driver.find_element(By.CLASS_NAME, 'password_input__field')
password_field.send_keys('4Flat4ever')
buttons = driver.find_elements(By.TAG_NAME, 'button')
buttons[1].click()

In [9]:
driver.get('https://www.carwow.co.uk/used-cars')

In [ ]:
brands_links = driver.find_elements(By.CLASS_NAME, 'multi-column-list__icon-text-link-wrap')

In [ ]:
for n in range(len(brands_links)):
    brands_links[n] = brands_links[n].get_attribute('href')

In [ ]:
brands = driver.find_elements(By.CSS_SELECTOR, 'span.multi-column-list__link-text')

In [ ]:
for n in range(len(brands)):
    brands[n] = brands[n].text[5:-5]

In [ ]:
driver.get(brands_links[0])

In [ ]:
more_button = driver.find_element(By.CLASS_NAME, 'product-filter__more-btn.btn.btn-secondary.btn-short')
more_button.click()
driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

In [ ]:
models = driver.find_elements(By.CLASS_NAME, 'card-generic__title')

In [ ]:
for n in range(len(models)):
    models[n] = models[n].text

In [ ]:
models_links = driver.find_elements(By.CLASS_NAME, 'btn.btn-action.btn-full-width.btn-short')

In [ ]:
for n in range(len(models_links)):
    models_links[n] = models_links[n].get_attribute('href')

In [ ]:
driver.get(models_links[0])

In [ ]:
link = driver.find_element(By.CLASS_NAME, 'btn.btn-action.btn-full-width.btn-short')
link.click()
driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

In [ ]:
total_cars = driver.find_element(By.CLASS_NAME, 'stock-filters-header-title__total')

In [ ]:
total_cars = int(total_cars.text.split()[-2])

In [ ]:
number_pages = driver.find_elements(By.CLASS_NAME, 'pagination__page')
number_pages = int(number_pages[len(number_pages)-2].find_element(By.TAG_NAME, 'a').text)

In [13]:
def numberOfCars(driver):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    driver.execute_script("window.scrollTo(document.body.scrollHeight, 0)")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    if len(cars_links) == 0:
        expected_no_links = 12
    elif (total_cars-len(cars_links)) >= 12:
        expected_no_links = 12
    else:
        expected_no_links = (total_cars-len(cars_links))
    return len(driver.find_elements(By.CLASS_NAME, 'btn.btn-action.btn-full-width.btn-short')) == expected_no_links

In [ ]:
i = 1
cars_links = []
car_variants = []
while i<=number_pages:  
    WebDriverWait(driver, timeout=10).until(numberOfCars)
    cars_link = driver.find_elements(By.CLASS_NAME, 'btn.btn-action.btn-full-width.btn-short')
    variants = driver.find_elements(By.CLASS_NAME, 'card-generic__title.card-generic__title--condensable')    
    for cars_lk in cars_link:
        cars_links.append(cars_lk.get_attribute('href'))
    for variant in variants:
        car_variants.append(variant.text)
    if(i != number_pages):
        next_button = driver.find_elements(By.CLASS_NAME, 'pagination__page')
        next_button[len(next_button)-1].click()
    i+=1

In [ ]:
driver.get(cars_links[0])

In [11]:
year_make = []
mileage = []
price = []
colour = []
transmission = []
fuel = []
for car_link in cars_links:
    driver.get(car_link)
    data = driver.find_elements(By.CLASS_NAME, 'summary-list__item')    
    year_make.append(data[0].find_element(By.TAG_NAME, 'dd').text)
    mileage.append(data[1].find_element(By.TAG_NAME, 'dd').text.split()[0])
    transmission.append(data[4].find_element(By.TAG_NAME, 'dd').text)
    fuel.append(data[5].find_element(By.TAG_NAME, 'dd').text)
    colour.append(data[7].find_element(By.TAG_NAME, 'dd').text)
    price.append(driver.find_element(By.CLASS_NAME, 'deal-pricing__carwow-price').text[1:])

In [ ]:
brand = []
model = []
for n in range(total_cars):
    brand.append(brands[0])
    model.append(models[0])

In [ ]:
df = pd.DataFrame(list(zip(brand, model, car_variants, price, year_make, mileage, transmission, fuel, colour)), 
                  columns=['brand', 'model', 'variant', 'price', 'year', 'mileage', 'transmission', 'fuel', 'colour'])

In [4]:
def numberOfCars(driver):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    driver.execute_script("window.scrollTo(document.body.scrollHeight, 0)")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    cars_link = driver.find_elements(By.CLASS_NAME, 'btn.btn-action.btn-full-width.btn-short')
    if total_cars <= 12:
        expected_no_links = total_cars
    elif len(cars_link)==0 and total_cars>=12:
        expected_no_links = 12
    elif (total_cars-len(cars_link)) >= 12:
        expected_no_links = 12
    else:
        expected_no_links = (total_cars-len(cars_link))
    return (len(driver.find_elements(By.CLASS_NAME, 'btn.btn-action.btn-full-width.btn-short')) >= expected_no_links)
def fetchData(driver):
    cars_link = driver.find_elements(By.CLASS_NAME, 'btn.btn-action.btn-full-width.btn-short')
    variants = driver.find_elements(By.CLASS_NAME, 'card-generic__title.card-generic__title--condensable')    
    for cars_lk in cars_link:
        cars_links.append(cars_lk.get_attribute('href'))
    for variant in variants:
        car_variants.append(variant.text)
    return True
driver.get('https://www.carwow.co.uk/used-cars')
brands_links = driver.find_elements(By.CLASS_NAME, 'multi-column-list__icon-text-link-wrap')
for i in range(len(brands_links)):
    brands_links[i] = brands_links[i].get_attribute('href')
brands = driver.find_elements(By.CSS_SELECTOR, 'span.multi-column-list__link-text')
for j in range(len(brands)):
    brands[j] = brands[j].text[5:-5]
for n in range(len(brands_links)):
    driver.get(brands_links[n])
    try:
        more_button = driver.find_element(By.CLASS_NAME, 'product-filter__more-btn.btn.btn-secondary.btn-short')
        more_button.click()
    except:
        print("ElementNotInteractableException Error")
    models = driver.find_elements(By.CLASS_NAME, 'card-generic__title')
    for k in range(len(models)):
        models[k] = models[k].text
    models_links = driver.find_elements(By.CLASS_NAME, 'btn.btn-action.btn-full-width.btn-short')
    for l in range(len(models_links)):
        models_links[l] = models_links[l].get_attribute('href')
    for m in range(len(models_links)):
        driver.get(models_links[m])
        try:
            link = driver.find_element(By.CLASS_NAME, 'btn.btn-action.btn-full-width.btn-short')
            link.click()
        except:
            continue
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        try:
            total_cars = driver.find_element(By.CLASS_NAME, 'stock-filters-header-title__total')
            total_cars = int(total_cars.text.split()[-2])
            try:
                driver.find_element(By.CLASS_NAME, 'pagination__page').is_displayed()
                number_pages = driver.find_elements(By.CLASS_NAME, 'pagination__page')
                number_pages = int(number_pages[len(number_pages)-2].find_element(By.TAG_NAME, 'a').text)
            except:
                number_pages = 1
            o = 1
            cars_links = []
            car_variants = []
            while o<=number_pages: 
                WebDriverWait(driver, timeout=60).until(numberOfCars)
                WebDriverWait(driver, timeout=60).until(fetchData)
                if(o != number_pages):
                    next_button = driver.find_elements(By.CLASS_NAME, 'pagination__page')
                    next_button[len(next_button)-1].click()
                o = o+1
            year_make = []
            mileage = []
            price = []
            colour = []
            transmission = []
            fuel = []
            for car_link in cars_links:
                driver.get(car_link)
                data = driver.find_elements(By.CLASS_NAME, 'summary-list__item')    
                year_make.append(data[0].find_element(By.TAG_NAME, 'dd').text)
                mileage.append(data[1].find_element(By.TAG_NAME, 'dd').text.split()[0])
                transmission.append(data[4].find_element(By.TAG_NAME, 'dd').text)
                fuel.append(data[5].find_element(By.TAG_NAME, 'dd').text)
                if (data[7].find_element(By.TAG_NAME, 'dt').text == 'Colour'):
                    colour.append(data[7].find_element(By.TAG_NAME, 'dd').text)
                else:
                    colour.append('Unknown')
                price.append(driver.find_element(By.CLASS_NAME, 'deal-pricing__carwow-price').text[1:])
            brand = []
            model = []
            for p in range(total_cars):
                brand.append(brands[n])
                model.append(models[m])
            temp = pd.DataFrame(list(zip(brand, model, car_variants, price, year_make, mileage, transmission, fuel, colour)), 
                      columns=['brand', 'model', 'variant', 'price', 'year', 'mileage', 'transmission', 'fuel', 'colour'])
            if n==0 and m==0:
                df = copy.deepcopy(temp)
            else:
                df = df.append(temp)
        except:
            continue

ElementNotInteractableException Error
ElementNotInteractableException Error
ElementNotInteractableException Error
ElementNotInteractableException Error
ElementNotInteractableException Error
ElementNotInteractableException Error
ElementNotInteractableException Error
ElementNotInteractableException Error
ElementNotInteractableException Error
ElementNotInteractableException Error
ElementNotInteractableException Error
ElementNotInteractableException Error
ElementNotInteractableException Error
ElementNotInteractableException Error
ElementNotInteractableException Error
ElementNotInteractableException Error
ElementNotInteractableException Error
ElementNotInteractableException Error
ElementNotInteractableException Error
ElementNotInteractableException Error
ElementNotInteractableException Error
ElementNotInteractableException Error
ElementNotInteractableException Error
ElementNotInteractableException Error
ElementNotInteractableException Error
ElementNotInteractableException Error
ElementNotIn

In [6]:
df.to_csv('carwow.csv')